# Python - intermediate

- profiling
- memory management
- socket
- threading
- async.io
- regular expression

## Profiling

https://docs.python.org/3/library/profile.html

In [1]:
import cProfile

def fibo(n):
    if n == 1:
        return 1
    if n == 2:
        return 1
    return fibo(n - 2) + fibo(n - 1)

# with cache
cache = {}

def fibo_with_cache(n):
    if n in cache:
        return cache[n]
    if n == 1:
        v = 1
    elif n == 2:
        v = 2
    else:
        v = fibo_with_cache(n - 2) + fibo_with_cache(n - 1)
    cache[n] = v
    return v

cProfile.run('fibo(30)')
cProfile.run('fibo_with_cache(30)')

         1664082 function calls (4 primitive calls) in 2.543 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
1664079/1    2.543    0.000    2.543    2.543 <ipython-input-1-9c9fe1d4bcdf>:3(fibo)
        1    0.000    0.000    2.543    2.543 <string>:1(<module>)
        1    0.000    0.000    2.543    2.543 {built-in method builtins.exec}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}


         60 function calls (4 primitive calls) in 0.000 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     57/1    0.000    0.000    0.000    0.000 <ipython-input-1-9c9fe1d4bcdf>:13(fibo_with_cache)
        1    0.000    0.000    0.000    0.000 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 {built-in method builtins.exec}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' object

## Memory management

In [2]:
print(id(-6))
print(id(-5))
print(id(1))
print(id(2-1))
print(id(256))
print(id(257))

140636021500016
10914304
10914496
10914496
10922656
140636021499984


In [3]:
print(id(-6))
print(id(-5))
print(id(1))
print(id(2-1))
print(id(256))
print(id(257))

140636021500080
10914304
10914496
10914496
10922656
140636021499216


In [4]:
import sys
print(sys.getrefcount(1))
a=1
print(sys.getrefcount(1))

3371
3372


In [5]:
class A:
    pass
print(id(A()))
print(id(A()))

140636021298008
140636021300976


In [6]:
a = A()
print(sys.getrefcount(a))
b = a
print(sys.getrefcount(a))

2
3


In [7]:
import gc
print("gc.isenabled()", gc.isenabled())
print("gc.get_threshold()", gc.get_threshold())
print("gc.get_count()", gc.get_count())

gc.isenabled() True
gc.get_threshold() (700, 10, 10)
gc.get_count() (448, 2, 3)


In [8]:
gc.collect()

0

In [9]:
print("gc.get_count()", gc.get_count())

gc.get_count() (335, 0, 0)


## Socket programming

In [25]:
import socket
import ssl

In [66]:
HOST = 'www.google.com'  # The server's hostname or IP address
PORT = 80                # The port used by the server

In [29]:
IP = socket.gethostbyname(HOST)
print(IP)

172.217.25.228


In [50]:
MESSAGE = \
b"""
GET / HTTP/1.1
User-Agent: Mozilla/4.0 (compatible; MSIE5.01; Windows NT)
Host: www.google.com
Accept-Language: en-us
Accept-Encoding: gzip, deflate
Connection: Keep-Alive

"""

In [67]:
with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.connect((HOST, PORT))
    s.sendall(MESSAGE)
    data = s.recv(1024*1024)

print(data.decode('UTF-8'))

HTTP/1.1 200 OK
Date: Tue, 22 Oct 2019 04:24:34 GMT
Expires: -1
Cache-Control: private, max-age=0
Content-Type: text/html; charset=UTF-8
P3P: CP="This is not a P3P policy! See g.co/p3phelp for more info."
Server: gws
X-XSS-Protection: 0
X-Frame-Options: SAMEORIGIN
Set-Cookie: 1P_JAR=2019-10-22-04; expires=Thu, 21-Nov-2019 04:24:34 GMT; path=/; domain=.google.com; SameSite=none
Set-Cookie: NID=189=i3f_0y6ZLdM4bPiM_ERJOF3YOzt0UvnHDu5wJw2gSrP9u31FdApVSXObS_d-8Rf4pXYvPCQYevQbr_eIrveGjmz96SEzSJUtx-HrAb2FUq5yomEh4ya-SAALmcarEaU8nkPECJRaYs1r_J7N6chwoXw_OdvJxENasnGfu71gcVs; expires=Wed, 22-Apr-2020 04:24:34 GMT; path=/; domain=.google.com; HttpOnly
Accept-Ranges: none
Vary: Accept-Encoding
Transfer-Encoding: chunked

152b
<!doctype html><html itemscope="" itemtype="http://schema.org/WebPage" lang="ko"><head><meta http-equiv="Content-Type" content="text/html; charset=UTF-8"><meta content="/images/branding/googleg/1x/googleg_standard_color_128dp.png" itemprop="image"><title>Google</title><style>

In [69]:
import socket
import ssl

PORT = 443         # The port used by the server

context = ssl.create_default_context()

with socket.create_connection((HOST, PORT)) as sock:
    with context.wrap_socket(sock, server_hostname=HOST) as ssock:
        ssock.sendall(MESSAGE)
        data = ssock.recv(1024*1024)

print(data.decode('UTF-8'))

HTTP/1.1 200 OK
Date: Tue, 22 Oct 2019 04:24:48 GMT
Expires: -1
Cache-Control: private, max-age=0
Content-Type: text/html; charset=UTF-8
P3P: CP="This is not a P3P policy! See g.co/p3phelp for more info."
Server: gws
X-XSS-Protection: 0
X-Frame-Options: SAMEORIGIN
Set-Cookie: 1P_JAR=2019-10-22-04; expires=Thu, 21-Nov-2019 04:24:48 GMT; path=/; domain=.google.com; SameSite=none
Set-Cookie: NID=189=bZJtBOJ4pWcLF2gZs8vmtI7wCmNf-WCbLpW4LwXJNxyn85cTiip7SEe4bhaZcPCtEm8qnMIbiVucCzu25FOPHAw6YQKQVj6rJ29GJFbIpat6mMGAdpzAyKj0KNSlPkdBIECIO53J9J8yDjN8L0JP0Jgmhc9dux5ZR1aJtzWkhOc; expires=Wed, 22-Apr-2020 04:24:48 GMT; path=/; domain=.google.com; HttpOnly
Alt-Svc: quic=":443"; ma=2592000; v="46,43",h3-Q048=":443"; ma=2592000,h3-Q046=":443"; ma=2592000,h3-Q043=":443"; ma=2592000
Accept-Ranges: none
Vary: Accept-Encoding
Transfer-Encoding: chunked

1533
<!doctype html><html itemscope="" itemtype="http://schema.org/WebPage" lang="ko"><head><meta http-equiv="Content-Type" content="text/html; charset=UTF-

## Threading

## asyncio

## Regular expressions

## References

- https://realpython.com/python-memory-management/
- https://stackify.com/python-garbage-collection/
- https://realpython.com/python-sockets/
- https://realpython.com/intro-to-python-threading/
- https://realpython.com/python-concurrency/